## Scenario: Implementing Payment Processing

A payment service needs to process orders using various payment methods (e.g., credit card, PayPal). The challenge is to design the system so that new payment methods can be added easily, and the code remains maintainable and adheres to SOLID principles.

In [ ]:
class PaymentService:
    def __init__(self, cost: int, include_delivery: bool):
        self.cost = cost
        self.include_delivery = include_delivery

    def process_order(self, payment_method: str):
        if payment_method == "CreditCard":
            card_number = "..."
            expiry_date = "..."
            cvv = "..."
            card = CreditCard(card_number, expiry_date, cvv)

            print("Validating credit card...")

            total = self.get_total()
            print(f"Paying {total} using Credit Card")
            card.set_amount(card.get_amount() - total)

        elif payment_method == "PayPal":
            email = "..."
            password = "..."

            print("Validating PayPal account...")

            total = self.get_total()
            print(f"Paying {total} using PayPal")
        else:
            print("Invalid payment method")

    def get_total(self) -> int:
        return self.cost + 10 if self.include_delivery else self.cost


class CreditCard:
    def __init__(self, card_number, expiry_date, cvv):
        self.amount = 1000
        self.card_number = card_number
        self.expiry_date = expiry_date
        self.cvv = cvv

    def set_amount(self, amount):
        self.amount = amount

    def get_amount(self):
        return self.amount


if __name__ == "__main__":
    payment_service = PaymentService(100, True)
    payment_service.process_order("CreditCard")
    payment_service = PaymentService(100, True)
    payment_service.process_order("PayPal")

## Problems with the Initial Implementation

The `PaymentService` class, as currently implemented, has several issues:

*   **Open/Closed Principle Violation:** Adding a new payment method (e.g., Apple Pay, Google Pay) requires modifying the `process_order` method by adding another `if` or `elif` block. This means the class is open for modification every time we want to support a new payment method, violating the Open/Closed Principle.
*   **Single Responsibility Principle Violation:** The `PaymentService` class is responsible for managing the payment process for multiple payment methods. This violates the Single Responsibility Principle, as the class has more than one reason to change.
*   **Code Duplication:** Logic for collecting and validating payment details might be duplicated across different payment methods.
*   **Maintainability Issues:** The `process_order` method becomes increasingly complex and harder to maintain as more payment methods are added.

To address the problems of the initial implementation, we need to:

*   **Separate Payment Methods:** Place each payment method into its own class, making that class responsible for a particular payment strategy.
*   **Interchangeable Strategies:** Ensure that these classes can be easily interchanged with one another, allowing the `PaymentService` to use different payment methods without modification.

# Strategy Pattern: Payment Processing

## Description

The Strategy pattern defines a family of algorithms, encapsulates each one into a separate class, and makes their objects interchangeable. This lets the algorithm vary independently of clients that use it. In this example, the algorithms are different payment methods (Credit Card, PayPal).

## Scenario

Building a payment processing system where different payment methods are supported. The client code (e.g., `PaymentService`) should be able to process payments without needing to know the specific payment method being used. This provides flexibility in adding new payment methods or changing existing ones without modifying the client code.

![Example](./image/example.png)

In [ ]:
from abc import ABC, abstractmethod

# Strategy Interface
class PaymentStrategy(ABC):
    @abstractmethod
    def collectPaymentDetails(self):
        pass

    @abstractmethod
    def validatePaymentDetails(self):
        pass

    @abstractmethod
    def pay(self, amount):
        pass

# Concrete Strategies
class PaymentByCreditCard(PaymentStrategy):
    def __init__(self):
        self.card_number = None
        self.expiry_date = None
        self.cvv = None

    def collectPaymentDetails(self):
        self.card_number = input("Enter card number: ")
        self.expiry_date = input("Enter expiry date: ")
        self.cvv = input("Enter CVV: ")

    def validatePaymentDetails(self):
        return True

    def pay(self, amount):
        print(f"Paying {amount} using Credit Card")

class PaymentByPayPal(PaymentStrategy):
    def __init__(self):
        self.email = None
        self.password = None

    def collectPaymentDetails(self):
        self.email = input("Enter PayPal email: ")
        self.password = input("Enter PayPal password: ")

    def validatePaymentDetails(self):
        return True

    def pay(self, amount):
        print(f"Paying {amount} using PayPal")

# Context
class PaymentService:
    def __init__(self):
        self.cost = 0
        self.include_delivery = False
        self.payment_strategy = None

    def set_payment_strategy(self, strategy: PaymentStrategy):
        self.payment_strategy = strategy

    def process_order(self):
        self.payment_strategy.collectPaymentDetails()
        if self.payment_strategy.validatePaymentDetails():
            total_amount = self.get_total()
            self.payment_strategy.pay(total_amount)
        else:
            print("Payment details are invalid.")

    def get_total(self):
        delivery_cost = 10 if self.include_delivery else 0
        return self.cost + delivery_cost

payment_service = PaymentService()
payment_service.cost = 100
payment_service.include_delivery = True

# Dynamically change payment strategy at runtime
payment_service.set_payment_strategy(PaymentByCreditCard())
payment_service.process_order()

payment_service.set_payment_strategy(PaymentByPayPal())
payment_service.process_order()

# Strategy Pattern: Payment Processing

## Scenario

A `PaymentService` needs to handle different payment methods (Credit Card, PayPal). A naive approach using `if/else` or `switch` statements to handle each payment method leads to code that is hard to maintain, violates the Open/Closed Principle, and the Single Responsibility Principle.

## Strategy Pattern Solution

The Strategy Pattern addresses these issues by:

1.  **Encapsulating Algorithms:** Defines a family of algorithms (payment methods), puts each into a separate class.
2.  **Interchangeability:** Makes these objects interchangeable, allowing the `PaymentService` to switch payment methods easily.

## Implementation Steps

1.  **Strategy Interface:** Defines a common interface (`PaymentStrategy`) for all payment methods. This interface includes the methods that the `PaymentService` will use (`collectPaymentDetails`, `validatePaymentDetails`, `pay`).
2.  **Concrete Strategies:** Implement the `PaymentStrategy` interface for each payment method (`PaymentByCreditCard`, `PaymentByPayPal`).
3.  **Context (PaymentService):**
    *   Maintains a reference to a `PaymentStrategy` object.
    *   Uses the strategy to process payments.
    *   *Does not* need to know the specific payment method being used.
4.  **Client:**
    *   Creates a specific strategy object.
    *   Passes it to the `PaymentService` via a setter.
    *   Can replace the strategy at runtime.

## Benefits

1.  **Open/Closed Principle:** Easy to add new payment methods without modifying existing code.
2.  **Single Responsibility Principle:** Each payment method is responsible for its own logic.
3.  **Code Reusability:** The payment algorithms are encapsulated and reusable.
4.  **Flexibility:** Payment method can be chosen and switched at runtime.
5.  **Decoupling:** `PaymentService` is decoupled from the specific payment implementations.                        |

## Key Takeaways

*   The Strategy pattern enables interchangeable algorithms by encapsulating them in separate classes.
*   It provides flexibility, promotes code reusability, and adheres to the Open/Closed and Single Responsibility Principles.
*   Use the Strategy pattern when you have multiple ways to perform a task and want to select the appropriate one at runtime.